In [3]:
import os

In [4]:
%pwd

'd:\\project_practice\\chicken_disease_classification_Project\\research'

In [5]:
os.chdir("../")

In [8]:
%pwd

'd:\\project_practice\\chicken_disease_classification_Project'

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir :Path

In [13]:
from cnn_Classifier.constants import *
from cnn_Classifier.utils.common import read_yaml, create_directories

In [33]:
class ConfigurationManager:
    def __init__(
        
        self,
        config_path = CONFIG_FILE_PATH,
        params_path = PARAMS_FILE_PATH):
                
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)

        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir

        )

        return data_ingestion_config

In [21]:
import os
import urllib.request as request
import zipfile
from cnn_Classifier import logger
from cnn_Classifier.utils.common import get_size

In [36]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config

        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download with following info:\n{headers}")
            
        else:
            logger.info(f"{filename} already exists of size {get_size(Path(self.config.local_data_file))}")


    def extract_zip_fle(self):

        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """

        unzip_path= self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=(True))
        with zipfile.ZipFile(self.config.local_data_file,"r") as zip_ref:
            zip_ref.extractall(unzip_path)

    

In [39]:
try:
    config= ConfigurationManager()
    data_ingestion_config= config.get_data_ingestion_config()
    data_ingestion= DataIngestion(config= data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_fle()

except Exception as e:
    raise e

[2024-01-18 13:01:38,520 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2024-01-18 13:01:38,523 : INFO : common : yaml file: params.yaml loaded successfully]
[2024-01-18 13:01:38,525 : INFO : common : created directory at: artifacts]
[2024-01-18 13:01:38,526 : INFO : common : created directory at: artifacts/data_ingestion]
[2024-01-18 13:01:40,802 : INFO : 1966388483 : artifacts/data_ingestion/data.zip download with following info:
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: D826:DF126:1DAF12:2C7056:65A8D3DA
Accept-Ranges: bytes
Date: Thu, 18 Jan 2024 07:31:39 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd1100030